## TF-Faster-RCNN-RPN

----------------------
I will implement RPN network in this notebook. 


#### Objective of this notebook 
1. Trian a rpn for object localization 


### Section list

1. Define a generator that return (1, height, width, 3), (1, nb_boxes, 4), (1, nb_boxes, 1)
2. Define the conv. layers 
3. Define the rpn layers 
4. Define the loss tensors
5. build network 
6. Train the network 

### Improt dependency  

In [17]:
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.contrib.layers.python.layers import regularizers
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope
import tensorflow.contrib.slim as slim
import tensorflow as tf
import os
import time

from scipy.misc import imresize

from math import floor,exp
import pprint

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pylab as plt
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import sklearn

### Define the global varables

In [2]:
anchor_box_scales = [128, 256, 512]
anchor_box_ratio = [[1,1],[1,2],[2,1]]
nb_anchors = len(anchor_box_scales) * len(anchor_box_ratio)
EPOCHES = 1

dataSets = ['VOC2012']

PRINT_TIME = False


TEST_FULL_IMG = np.array([mpimg.imread("./test1.jpg")])
print(TEST_FULL_IMG.shape)
print(TEST_FULL_IMG.shape[1]/4, TEST_FULL_IMG.shape[2]/4)

(1, 720, 1280, 3)
180.0 320.0


## Define generator

We use VOC datasets in this implmentation.

### Get annotations 

In [3]:

def get_voc_data(input_path):
    all_imgs = []

    classes_count = {}

    class_mappingNameToId = {}
    class_mappingIdToName = {}

    visualise = False

    data_paths = [os.path.join(input_path, s) for s in dataSets]

    print ('Parsing annotation files')

    for data_path in data_paths:

        annot_path = os.path.join(data_path, 'Annotations')
        imgs_path = os.path.join(data_path, 'JPEGImages')
        imgsets_path_trainval = os.path.join(data_path, 'ImageSets',
                'Main', 'trainval.txt')

# ........imgsets_path_test = os.path.join(data_path, 'ImageSets','Main','test.txt')

        trainval_files = []
        test_files = []
        try:
            with open(imgsets_path_trainval) as f:
                for line in f:
                    trainval_files.append(line.strip() + '.jpg')
        except Exception as e:

# ............with open(imgsets_path_test) as f:
# ................for line in f:
# ....................test_files.append(line.strip() + '.jpg')

            print (e)

        annots = [os.path.join(annot_path, s) for s in
                  os.listdir(annot_path)]
        idx = 0
        for annot in annots:
            try:
                idx += 1

                et = ET.parse(annot)
                element = et.getroot()

                element_objs = element.findall('object')
                element_filename = element.find('filename').text
                element_width = int(element.find('size').find('width'
                                    ).text)
                element_height = int(element.find('size').find('height'
                        ).text)

                if len(element_objs) > 0:
                    annotation_data = {
                        'filepath': os.path.join(imgs_path,
                                element_filename),
                        'width': element_width,
                        'height': element_height,
                        'bboxes': [],
                        }

                    if element_filename in trainval_files:
                        annotation_data['imageset'] = 'trainval'
                    elif element_filename in test_files:
                        annotation_data['imageset'] = 'test'
                    else:
                        annotation_data['imageset'] = 'trainval'

                for element_obj in element_objs:
                    class_name = element_obj.find('name').text
                    if class_name not in classes_count:
                        classes_count[class_name] = 1
                    else:
                        classes_count[class_name] += 1

                    if class_name not in class_mappingNameToId:
                        class_mappingNameToId[class_name] = \
                            len(class_mappingNameToId)
                        class_mappingIdToName[len(class_mappingNameToId)
                                - 1] = class_name

                    obj_bbox = element_obj.find('bndbox')
                    x1 = int(round(float(obj_bbox.find('xmin').text)))
                    y1 = int(round(float(obj_bbox.find('ymin').text)))
                    x2 = int(round(float(obj_bbox.find('xmax').text)))
                    y2 = int(round(float(obj_bbox.find('ymax').text)))
                    w = x2 - x1
                    h = y2 - y1
                    x = int(round(x1 + w / 2))
                    y = int(round(y1 + h / 2))
                    difficulty = int(element_obj.find('difficult'
                            ).text) == 1
                    annotation_data['bboxes'].append({
                        'class': class_name,
                        'x1': x1,
                        'x2': x2,
                        'y1': y1,
                        'y2': y2,
                        'difficult': difficulty,
                        'x': x,
                        'y': y,
                        'w': w,
                        'h': h,
                        })
                all_imgs.append(annotation_data)

                if visualise:
                    img = cv2.imread(annotation_data['filepath'])
                    for bbox in annotation_data['bboxes']:
                        cv2.rectangle(img, (bbox['x1'], bbox['y1']),
                                (bbox['x2'], bbox['y2']), (0, 0, 255))
                    cv2.imshow('img', img)
                    cv2.waitKey(0)
            except Exception as  e:

                print (e)
                continue

        # make if no bg in the className make bg class

        if 'bg' not in classes_count:
            print ('bg not in class')
            
            class0Name = class_mappingIdToName[0]
            class0NewId = len(class_mappingNameToId)
            
            
            classes_count['bg'] = 0
            class_mappingNameToId['bg'] = 0 
            class_mappingIdToName[0] = 'bg'
            
            class_mappingNameToId[class0Name] = class0NewId
            class_mappingIdToName[len(class_mappingNameToId)-1] = class0Name
            
            
        else:
            print ('bg in class')
            # if there are a bg class make it first index
            bgOldId = class_mappingNameToId['bg']
            #switch id bg to 0id class
            if bgOldId != 0 :
                class0Name = class_mappingIdToName[0]
                class0NewId = bgOldId
                
                class_mappingIdToName[0] = 'bg'
                class_mappingNameToId['bg'] = 0
                
                class_mappingIdToName[class0NewId] = class0Name
                class_mappingNameToId[class0Name] = class0NewId
            
    return (all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName)

In [4]:
class Data_source:
    def __init__(self):
        all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName = get_voc_data("./data")
        self.all_imgs = all_imgs
        self.classes_count = classes_count
        self.class_mappingNameToId = class_mappingNameToId
        self.class_mappingIdToName = class_mappingIdToName
        
    def get_classId(self, class_name):
        return self.class_mappingNameToId[class_name]
    
    def get_total_imgs(self):
        return len(self.all_imgs)
    
    def get_generator(self):
        '''
            generator use in training
            imgs 
            x,y,w,h
            x1,y1,x2,y2
            class
            
            return (1, height, width, 3), (1, nb_boxes, 4), (1, nb_boxes, 4), (1, nb_boxes, 1)
        '''
        records = sklearn.utils.shuffle(self.all_imgs)
        for record in records:
            img = mpimg.imread(record["filepath"])
            
            box_xywh = []
            box_xyxy = []
            box_class = []
            
            for box in record["bboxes"]:
                box_xywh.append([box["x"], box["y"], box["w"], box["h"]])
                box_xyxy.append([box["x1"], box["y1"], box["x2"], box["y2"]])
                name = self.get_classId(box["class"])
                box_class.append([name])
        
            yield np.array([img]), np.array([box_xywh]), np.array([box_xyxy]), np.array([box_class])
    

In [5]:
datas = Data_source()
generator = datas.get_generator()

Parsing annotation files
bg not in class


In [6]:

img, xtwh, xxyy, cls = next(generator)

## Define the conv. layers

---------

I will define vgg16 conv layers. 

Vgg 16 used in this project, it use the pretrain network from imageNet. The network defined until conv5.

The vgg16 function accept iamge input and otput the nets tensor and the enpoints.

We use conv5 as our feature map layer, which will port to rpn and rcnn.

In [7]:
# Define the vgg16 layers
def vgg_16(inputs,  scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'

        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
#             net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
#             net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return net, end_points

## Define the rpn layers


we will define the RPN network. 

It have few steps. 
1. Define the conv layers of rpn.
2. Mapping rpn to bbox

In [8]:

def rpn_cls_shape_func(in_list):
    cls = in_list
    return np.array([cls.shape[0], cls.shape[1],cls.shape[2],cls.shape[3],1]).astype(np.int32)

def rpn_regr_shape_func(in_list):
    regr = in_list
    return np.array([regr.shape[0], regr.shape[1],regr.shape[2],regr.shape[3]/4,4]).astype(np.int32)

def rpn(net, num_anchors=9, scope="rpn"):
    with tf.variable_scope(scope, 'rpn', [net]) as sc:
        end_points_collection = sc.name + '_end_points'
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], 
                            outputs_collections=end_points_collection, 
                            activation_fn=tf.nn.relu,
                            weights_initializer=tf.truncated_normal_initializer(0.0, 0.01)):
            
            net = slim.conv2d(net, 512, [3, 1], scope='rpn_conv_3x3', padding='SAME')
            
            rpn_class = slim.conv2d(net, num_anchors, [1, 1], scope='rpn_class')
            rpn_cls_shape = tf.py_func(rpn_cls_shape_func, [rpn_class], tf.int32, name="rpn_cls_shape")
            rpn_class = tf.reshape(rpn_class, rpn_cls_shape)
            
            rpn_regr = slim.conv2d(net, num_anchors*4, [1, 1], scope='rpn_regr')  
            rpn_regr_shape = tf.py_func(rpn_regr_shape_func, [rpn_regr], tf.int32, name="rpn_regr_shape")
            rpn_regr = tf.reshape(rpn_regr, rpn_regr_shape)

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return rpn_class, rpn_regr, end_points

#### Define proposal layer 

1. The network will tranform the network result to the x,y,w,h 
2. Then it will transfrom to x1,y1,x2,y2 format 
3. take top N proposals eg: 6000
4. Non maximum supreesion
5. take top N proposal 300

---------

In [9]:
def transform_txtytwth_to_xywh(rpn_regr, anchor_box):
    '''apply clipping box'''
     
    pass
def transform_xywh_to_xyxy(xywh):
    '''transform the format '''
    pass
def combine_into_one_list(xyxy, cls):
    '''combine cls and regr res into one list, which faster for calculate the res'''
    pass
def select_top_6000(bbox):
    '''select 6000 bbox'''
    pass
def apply_nms():
    ''' apply nms '''
    pass
def select_top_300(bbox):
    ''' select first 300 bbox'''
    pass
def rpn_proposal_layer(anchors, rpn_regr, rpn_cls):
    ''' used when buidl the network '''
    pass

#### Generate ground truth

In [10]:

def create_generate_anchors( anchor_box_scales = [128, 256, 512],   anchor_box_ratio = [[1,1],[1,2],[2,1]] ):
    def generate_anchors(featureMap, img):
        '''
            Not batch is return, which can save memory in run time 
        '''
        
        start = time.time()
        
        stepSize = int(img.shape[1]/featureMap.shape[1])
        imgWidth = img.shape[2]
        imgheight = img.shape[1]
        
        anchors = []
        for scale in anchor_box_scales:
            for ratio in anchor_box_ratio:
                anchors.append([ratio[0]*scale,ratio[1]*scale])
        anchors = np.array(anchors)

        bbox = np.zeros((featureMap.shape[1], featureMap.shape[2], len(anchors), 4))
        
        if(featureMap.shape[2] != int(imgWidth/4) ):
            print("---------- featureMap.shape[2]",featureMap.shape[2],"imgWidth/4",imgWidth/4, "int(imgWidth/4)",int(imgWidth/4))

#         base on the feature map that input to this function 
        x = range(int(stepSize/2), featureMap.shape[2]*stepSize, stepSize)
        y = range(int(stepSize/2), featureMap.shape[1]*stepSize, stepSize)

        xv, yv= np.meshgrid(x, y)

        for anchorIdx, width, height in zip(range(len(anchors)), anchors[:,0], anchors[:,1]):
            bbox[:,:,anchorIdx,0] = xv
            bbox[:,:,anchorIdx,1] = yv
            bbox[:,:,anchorIdx,2].fill(width)
            bbox[:,:,anchorIdx,3].fill(height)
        bbox = bbox.astype(np.int32)
        
        if(PRINT_TIME):
            print("generate_anchors use",time.time() - start,"s")

        return bbox
    return generate_anchors


In [11]:

def union(au, bu):
    x = min(au[0], bu[0])
    y = min(au[1], bu[1])
    w = max(au[2], bu[2]) - x
    h = max(au[3], bu[3]) - y
    return x, y, w, h

def intersection(ai, bi):
    x = max(ai[0], bi[0])
    y = max(ai[1], bi[1])
    w = min(ai[2], bi[2]) - x
    h = min(ai[3], bi[3]) - y
    if w < 0 or h < 0:
        return 0, 0, 0, 0
    return x, y, w, h

def iou(a_, b_):
#   a, b should be x,y,w,h format
    # a and b should be (x1,y1,x2,y2)
    x1 = a_[0] - a_[2]/2
    y1 = a_[1] - a_[3]/2
    a = [x1, y1, x1+a_[2], y1 + a_[2]]
    
    x1 = b_[0] - b_[2]/2
    y1 = b_[1] - b_[3]/2
    b = [x1, y1, x1+b_[2], y1 + b_[2]]
    
    if a[0] >= a[2] or a[1] >= a[3] or b[0] >= b[2] or b[1] >= b[3]:
        return 0.0

    i = intersection(a, b)
    u = union(a, b)

    area_i = i[2] * i[3]
    area_u = u[2] * u[3]
    return float(area_i) / float(area_u)

#### Test IoU

In [12]:
def testIoU():
    box_a = [50,50,50,50]
    box_b = [50,55,50,70]
    v = iou(box_a, box_b)
    assert( v> 0.7)
testIoU()

In [13]:
def cal_parameterizations_bbox(ax, ay, aw, ah, bx, by, bw, bh):
    return [(ax-bx)/bw, (ay-by)/bh, exp(aw/bw), exp(ah/bh)]


def create_rpn_ground_truth(anchors, bboxes, bboxes_cls):
    '''
        Since it is training one img is accepted 
        input shape = (1,None,4) in x,y,w,h
        return anchorbox with t_x, t_y, t_w, t_h and anchorBoxLabel with -1 neg, 1 is pos, 0 is nth
    '''
    start = time.time()
    
    regr_y = np.zeros(anchors.shape)
    cls_shape = np.array(anchors.shape)
    cls_shape[-1] = 1
    cls_y = np.zeros(cls_shape)
    
#     print("cls_shape",cls_shape)
#     print("anchors.shape",anchors.shape)
#     print("bboxes.shape",bboxes.shape)
#     print("bboxes_cls.shape",bboxes_cls.shape)
    
    highest_iou = (0.,[])
    debugeMsg = 0
    
    for box, box_class in zip(bboxes[0],bboxes_cls[0]):
        # skip bg class
        if box_class == 0:
            continue
        pos_list = []
        neg_list = []
        
        start_cal_iou = time.time()
        for y in range(anchors.shape[0]):
            for x in range(anchors.shape[1]):
                for a in range(anchors.shape[2]):
                    anchor = anchors[y,x,a]
                    regr   = regr_y[y,x,a]
                    cls    = cls_y[y,x,a]
                    
                    # cal the iou 
                    iou_v = iou(anchor, box)
                    debugeMsg = iou_v
                    
                    if iou_v > highest_iou[0]:
                        highest_iou = (iou_v,[y,x,a])
                        
                    # if iou <= 0.3
#                        add to neg list
                    if iou_v <= 0.3:
#                         if(abs(anchor[0]-box[0]) < 50 and abs(anchor[1]-box[1]) < 50):
#                             print("anchor, box, iou_v",anchor, box, iou_v)
                        neg_list.append([y,x,a])
                    # if iou >= 0.7
#                        add to pos list
                    elif iou_v >= 0.7:
                        pos_list.append([y,x,a])
        if(PRINT_TIME):
            print("cal_iou use",time.time() - start_cal_iou,"s")
                                
                    
#         print("highest_iou",highest_iou)
#         print("len(pos_list)",len(pos_list))
#         print("len(neg_list)",len(neg_list))
#         print("box",box)
        
        
        if len(pos_list) == 0:
            pos_list.append(highest_iou[1])
            
            
        pos_list = np.array(pos_list)
        neg_list = np.array(neg_list)
        
        
#       filter max 256 or balance the smapling in ratio 1:1       
        smaple_size = min(len(pos_list), len(neg_list), 256)
    
#         print("min(len(pos_list), len(neg_list), 256)", len(pos_list), len(neg_list), 256)
        idx_pos = np.random.randint(len(pos_list), size=smaple_size)
        idx_neg = np.random.randint(len(neg_list), size=smaple_size)
            
#         print("len(idx_pos)",len(idx_pos))
#         print("len(idx_neg)",len(idx_neg))

#       label the cls and cal. the parameterizations 
        selected_pos = pos_list[idx_pos]
#         if len(selected_pos) == 1 or highest_iou[0] < 0.1:
#             print("selected_pos ==1", selected_pos)
#             print("idx_pos",idx_pos)
#             print("pos_list",pos_list)
#             print(highest_iou)
#             print(debugeMsg)
#             print(box)
#         print("selected_pos",selected_pos)
#         print("idx_pos",idx_pos)
        for r_pos in selected_pos:
            y = r_pos[0]
            x = r_pos[1]
            a = r_pos[2]
            anchor = anchors[y,x,a]
            regr_y[y,x,a] = cal_parameterizations_bbox(box[0], box[1], box[2], box[3], anchor[0], anchor[1], anchor[2], anchor[3])
            cls_y[y,x,a] = 1
        
        selected_neg = neg_list[idx_neg]
#         print("selected_neg",selected_neg)
        for r_neg in selected_neg:
            y = r_neg[0]
            x = r_neg[1]
            a = r_neg[2]
            anchor = anchors[y,x,a]
            regr_y[y,x,a] = cal_parameterizations_bbox(box[0], box[1], box[2], box[3], anchor[0], anchor[1], anchor[2], anchor[3])
            cls_y[y,x,a] = -1
        
#   select highest iou if none in the pos list            
#     print(cls_y)            
    if(PRINT_TIME):
        print("create_rpn_ground_truth use",time.time() - start,"s")
    return regr_y.astype(np.float32), cls_y.astype(np.float32)
 
def cls_ground_truth(regr, cls):
    return cls

def regr_ground_truth(regr, cls):
    return regr  

## Define the loss tensors


In [14]:
lambda_rpn_regr = 1.0
lambda_rpn_class = 1.0


esilon = 1e-4


def smooth_l1(bbox_pred, bbox_targets, name=""):
    """
        ResultLoss = outside_weights * SmoothL1(inside_weights * (bbox_pred - bbox_targets))
        SmoothL1(x) = 0.5 * ( x)^2,    if |x| < 1
                      |x| - 0.5 ,    otherwise
    """
    with tf.variable_scope("smooth_l1_"+name):
        x = tf.subtract(bbox_pred, bbox_targets)

        smooth_l1_sign = tf.cast(tf.less(tf.abs(x), 1.0 ), tf.float32)
        smooth_l1_option1 = tf.multiply(tf.multiply(x, x), 0.5 )
        smooth_l1_option2 = tf.subtract(tf.abs(x), 0.5 )
        smooth_l1_result = tf.add(tf.multiply(smooth_l1_option1, smooth_l1_sign),
                                  tf.multiply(smooth_l1_option2, tf.abs(tf.subtract(smooth_l1_sign, 1.0))))

    return smooth_l1_result


def rpn_regr_loss(y_pred, y_true_regr, y_true_cls):
    
    epsilon_tf = tf.constant(esilon, dtype=tf.float32, name="epsilon_tf")
    x = y_pred - y_true_regr 
    x_abs = tf.abs(x)
    x_bool = tf.cast(tf.less_equal(x_abs, 1.0), tf.float32, name="x_bool")
    
    x_smooth_l1 = (x_bool * (0.5 * x * x) + (1 - x_bool) * (x_abs - 0.5))
    x_smooth_l1_pos_only = y_true_cls * x_smooth_l1
    
    return lambda_rpn_regr * x_smooth_l1_pos_only / tf.reduce_sum(epsilon_tf + y_true_cls)
    
def rpn_class_loss(y_pred, y_true_cls):    
    epsilon_tf = tf.constant(esilon, dtype=tf.float32, name="epsilon_tf")
    return tf.nn.softmax_cross_entropy_with_logits( labels=y_true_cls,  logits=y_pred, name="softmax_cross_entropy_with_logits")
#     x = y_true_cls * tf.binary_crossentropy(y_pred, y_true_cls)
#     return lambda_rpn_class * tf.sum(x) / tf.reduce_sum(epsilon_tf + y_true_cls)

## Build network

In [15]:
img_input = tf.placeholder(tf.float32, [1, None, None, 3])

# input the bbox location and the class id 
# in format x,y,w,h,classId
y_bbox_regr = tf.placeholder(tf.float32, [1, None, 4])

y_bbox_cls = tf.placeholder(tf.float32, [1, None, 1])

conv_layer,conv_end_points = vgg_16(img_input)
conv_restore_names = [ item for item in conv_end_points] 

rpn_class, rpn_regr, rpn_end_points = rpn(conv_layer,nb_anchors)
rpn_class_softmax = tf.nn.softmax(rpn_class)

generate_anchors = tf.py_func(create_generate_anchors(), [rpn_regr, img_input], tf.int32, name="generate_anchors")

# generate grounth rpn regr. truth 
rpn_ground_truth = tf.py_func(create_rpn_ground_truth, [generate_anchors, y_bbox_regr, y_bbox_cls], [tf.float32, tf.float32], name="rpn_ground_truth")


rpn_cls_ground_truth  = tf.py_func(cls_ground_truth,  rpn_ground_truth, tf.float32, name="rpn_cls_ground_truth")
rpn_regr_ground_truth = tf.py_func(regr_ground_truth, rpn_ground_truth, tf.float32, name="rpn_regr_ground_truth")


# restore weights
variables_to_restore = slim.get_variables_to_restore(include=conv_restore_names, exclude=['rpn_adam'])
vgg_checkpoint_path = os.path.join("./", 'vgg_16.ckpt')

restorer = tf.train.Saver(variables_to_restore)
    
rpn_loss_regr = tf.reduce_mean(rpn_regr_loss(rpn_regr, rpn_regr_ground_truth, rpn_cls_ground_truth))
rpn_loss_cls = tf.reduce_mean(rpn_class_loss(rpn_class, rpn_cls_ground_truth))
rpn_loss = rpn_loss_regr + rpn_loss_cls

tf.summary.scalar('rpn_loss', rpn_loss)
tf.summary.scalar('rpn_loss_cls', rpn_loss_cls)
tf.summary.scalar('rpn_loss_regr', rpn_loss_regr)

rpn_optimizer = tf.train.AdamOptimizer(0.005, name='rpn_adam').minimize(rpn_loss)

init_op = tf.global_variables_initializer()
merged = tf.summary.merge_all()

with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "./vgg_16.ckpt")

    sess.run(init_op)
    print("restore conv layers")
    
    print("init tensorboard ")
    train_writer = tf.summary.FileWriter("logs/train", sess.graph)  
    
    datas_source = Data_source()
    totalNbImgs = datas_source.get_total_imgs()    
    
    # epoches 
    for epoch in range(EPOCHES):
        generator = datas_source.get_generator()
        
        curImg = 1
#       loop images in one epoch 
        for img, xywh, xxyy, cls in generator:
        
            start = time.time()
            
            target_tensor = [rpn_optimizer, merged]
            feed_dict = {
                img_input:   img,
                y_bbox_regr: xywh,
                y_bbox_cls:  cls
            }

            opt, hist = sess.run(target_tensor, feed_dict=feed_dict)
            
            if curImg%1 == 0:
                print("epoch " +str(epoch+1)+" "+ str(curImg)+"/"+str(totalNbImgs))
            
            if(PRINT_TIME):
                print(str(curImg), "str(curImg) use",time.time() - start,"s")
            
            curImg+=1
            if(curImg > 600):
                break
    
tf.reset_default_graph()

restore conv layers
init tensorboard 
Parsing annotation files
bg not in class
epoch 1 1/17125
epoch 1 2/17125
epoch 1 3/17125
epoch 1 4/17125
epoch 1 5/17125
epoch 1 6/17125
epoch 1 7/17125
epoch 1 8/17125
epoch 1 9/17125
epoch 1 10/17125
epoch 1 11/17125
epoch 1 12/17125
epoch 1 13/17125
epoch 1 14/17125
epoch 1 15/17125
epoch 1 16/17125
epoch 1 17/17125
epoch 1 18/17125
epoch 1 19/17125
epoch 1 20/17125
epoch 1 21/17125
epoch 1 22/17125
epoch 1 23/17125
epoch 1 24/17125
epoch 1 25/17125
epoch 1 26/17125
epoch 1 27/17125
epoch 1 28/17125
epoch 1 29/17125
epoch 1 30/17125
epoch 1 31/17125
epoch 1 32/17125
epoch 1 33/17125
epoch 1 34/17125
epoch 1 35/17125
epoch 1 36/17125
epoch 1 37/17125
epoch 1 38/17125
epoch 1 39/17125
epoch 1 40/17125
epoch 1 41/17125
epoch 1 42/17125
epoch 1 43/17125
epoch 1 44/17125
epoch 1 45/17125
epoch 1 46/17125
epoch 1 47/17125
epoch 1 48/17125
epoch 1 49/17125
epoch 1 50/17125
epoch 1 51/17125
epoch 1 52/17125
epoch 1 53/17125
epoch 1 54/17125
epoch 1 55/1

In [16]:
tf.reset_default_graph()
